In [13]:
import os
import torch
import torch.nn as nn
from torchlight.io import IO
import numpy as np
from collections import OrderedDict
from torchlight.io import IO
import cv2
# From Python
# It requires OpenCV installed for Python
import sys
import cv2
import os
dir_path = r'D:\Projects\anomaly\openpose_full\openpose\build'
sys.path.append(dir_path + r'\python\openpose\Release')
os.environ['PATH']  = os.environ['PATH'] + ';' + dir_path + r'\x64\Release;' +  dir_path + r'\bin;'
model_path = r'D:\Projects\anomaly\openpose_full\openpose\models'
import pyopenpose as op
import numpy as np

import time
    

# try:
#    
params = dict()
params["model_folder"] = model_path
params["number_people_max"] = 1
params["model_pose"] = "COCO"
params["scale_number"] = 4
# params["net_resolution"] = "-1x256"
# params["face"] = False
# params["hand"] = False


#     # Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()


In [3]:
model_name = 'net.st_gcn.Model'
graph_args = {'layout':'openpose', 'strategy':'spatial'}
model_args = {
    'edge_importance_weighting':True,
    'graph_args':graph_args,
    'in_channels':3,
    'num_class':400, }
weights_path =  './models/st_gcn.kinetics.pt'

io = IO('./')
model = io.load_model(model_name, **model_args).to('cuda')
# Need to turn into a func
weights = torch.load(weights_path)
weights_list = [[k.split('module.')[-1], v.cpu()] for k, v in weights.items()]
weights = OrderedDict(weights_list)
model_state_dict = model.state_dict()
weights.pop('fcn.bias') # loading all but the Final FC layer's weight and bias
weights.pop('fcn.weight')
model_state_dict.update(weights)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [4]:
# load torch model .pth

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(in_features=18 * 75 * 256, out_features=1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.Linear(256, 18 * 75 * 256),
            
        )

    def forward(self, x):
        # print('input shape', x.shape)
        x = x.reshape(x.size(0), -1)
        # print('flattened shape', x.shape)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(x.size(0), 18, 75, 256)  # Reshape back to the original size
        return x
    
testModel = Autoencoder().to('cuda')
testModel.load_state_dict(torch.load('./model/autoencoder_torch_1126_T0.7_c0.8.pth'))
testModel.eval()


frameNum = 30
threshold =0.7
mse_cutoff = 0.85
def getParams():
    return frameNum, threshold, mse_cutoff


def predictFall(skeletonLoader, width, height):    
    _,residual_threshold, mse_cutoff= getParams()
    # print (len(skeletonLoader))

    skelNumpy = np.array(skeletonLoader)
    # print(skelNumpy.shape)
    skelNumpy = skelNumpy.reshape(skelNumpy.shape[0], 18, 3)


    padArray = np.zeros((300-skelNumpy.shape[0], 18, 3))
    skelNumpy = np.concatenate((skelNumpy, padArray), axis=0)
    # # 18 300 3
    # # transpose skeletonNum to (3, 300, 18) 
    # then reshape skeletonNum to (1, 3, 300, 18, 1)
    skelNumpy = skelNumpy.transpose(2, 0, 1).reshape(1, 3, 300, 18, 1)
    skelNumpy = torch.from_numpy(skelNumpy).float()

    _, feature = model.extract_feature(skelNumpy.to('cuda'))

    # print(feature[1].shape) 
    
    feature = feature.cpu().detach().numpy()
    feature = feature.reshape(1, 18, 75, 256)
    
    result= testModel(torch.from_numpy(feature).to('cuda')).cpu().detach().numpy()

    residual = np.abs(feature - result)
    residual=residual- residual_threshold

    mse = np.mean(np.power(residual, 2))
    # print (mse)

    return mse >= mse_cutoff, mse



In [11]:

def runFolder(folder):
    STOP = False

    skeletonLoader = [] 
    
    fallResult = []
    imagesFile=[ f for f in os.listdir(folder) if f.endswith('.png') or f.endswith('.jpg')]
    imagesFile.sort()
    imagesFile=[os.path.join(folder, f) for f in imagesFile]
    
    winTitle = 'Folder: '+folder
    for j in range(len(imagesFile)):
        # current_time = time.time()
        startRecordTime = time.time()
        datum = op.Datum()

        # print(images[i])
        img = cv2.imread(imagesFile[j])
        # resize image
        # img = cv2.resize(img, (320, 176))
        height, width, channels = img.shape
    
        datum.cvInputData = img
        opWrapper.emplaceAndPop(op.VectorDatum([datum]))
        # timeDiff = time.time()-startRecordTime
        # print (timeDiff, 'OPENPOSE')

        if datum.poseKeypoints is None:
            cv2.imshow(winTitle, img)
            cv2.waitKey(1000//1000) 
            # print ('No skeleton')
            continue

        x = datum.poseKeypoints[:,:,0]
        y = datum.poseKeypoints[:,:,1]

        x=x[x!=0]
        y=y[y!=0]

        min_x = x.min()-15
        min_y = y.min()-10
        max_x = x.max()+10
        max_y = y.max()+15

        
        # normalize to width and height
        datum.poseKeypoints[:,:,0]=datum.poseKeypoints[:,:,0]/width-0.5
        datum.poseKeypoints[:,:,1]=datum.poseKeypoints[:,:,1]/height-0.5
        pose_keypoints=np.array(datum.poseKeypoints)
        # print (pose_keypoints.shape)# (1, 18, 3)
        pose_keypoints=pose_keypoints.reshape(18,3)

    
        if len(skeletonLoader) >= 300:        
                # remove the first frame 
            skeletonLoader=skeletonLoader[10:]
        skeletonLoader.append(datum.poseKeypoints)
        


        frameNum, _, _= getParams()
        fall= False
        mse=0
        if len(skeletonLoader) >=frameNum and len(skeletonLoader)%10 == 0:
            startRecordTime = time.time()
            fall, mse=predictFall(skeletonLoader, width, height)
            # timeDiff = time.time()-startRecordTime
            # print (timeDiff, 'PREDICT FALL')
            if len(fallResult) >=2:
                fallResult=fallResult[1:]
            fallResult.append(fall)
            print (fall, mse) 
        # else:
        #     print ('continue')

        color = (0,255,0)
        text = 'ADL'
        if fallResult.count(True) >=1:
            color = (0,0,255)
            text = 'FALL'


        
        # draw red frame in cv2 img
        img = cv2.rectangle(img, (int(min_x), int(min_y)), (int(max_x), int(max_y)), color, 2)
        # draw text in cv2 img
        img = cv2.putText(img, text, (int(min_x), int(min_y)-5), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        cv2.imshow(winTitle, img)
        # save image
        # ouputFolder = r'd:\output'+'\\'+folder.split('/')[-1]
        # print (ouputFolder)
        # if not os.path.exists(ouputFolder):
        #     os.makedirs(ouputFolder)
        # file=os.path.join(ouputFolder, imagesFile[j].split('\\')[-1].split('.')[0]+'.jpg')
        # print (file)
        # cv2.imwrite(file, img)
        if cv2.waitKey(1000//1000) & 0xFF == ord('q'):
            STOP = True
            break
    cv2.destroyAllWindows()
    return STOP

In [15]:
testFold='./data/urfall/images/fall'
subfolders = [ f.path for f in os.scandir(testFold) if f.is_dir() ]
for subfolder in subfolders:
    print ('Fall folder: ', subfolder)
    if runFolder(subfolder):
        break
    
testFold='./data/urfall/images/adl'
subfolders = [ f.path for f in os.scandir(testFold) if f.is_dir() ]
for subfolder in subfolders:
    print ('ADL folder: ', subfolder)
    if runFolder(subfolder):
        break



    


Fall folder:  ./data/urfall/images/fall\fall-01-cam0-rgb
False 0.6435527
False 0.6413512
False 0.6407867
True 0.8500412
True 0.85943794
True 0.9532546
True 0.9416906
True 0.8634365
True 0.97037596
Fall folder:  ./data/urfall/images/fall\fall-02-cam0-rgb
False 0.71314067
False 0.75487787
False 0.8362087
True 0.8836672
True 1.009912
True 1.0082698
True 0.98337674
Fall folder:  ./data/urfall/images/fall\fall-03-cam0-rgb
False 0.7849506
False 0.7582361
False 0.733239
False 0.7105757
False 0.7213396
False 0.7733422
False 0.7899255
False 0.77409285
True 0.87274235
True 0.87871474
True 0.90037966
True 0.9451649
True 0.94018185
True 0.9884813
True 1.0130416
True 0.98296386
True 0.94493717
True 0.9556
True 0.95600605
Fall folder:  ./data/urfall/images/fall\fall-04-cam0-rgb
True 1.067128
True 1.1218171
True 1.0799009
True 1.0360729
Fall folder:  ./data/urfall/images/fall\fall-05-cam0-rgb
True 1.0843365
True 1.0986197
True 1.0836134
True 1.0292859
True 0.9864233
True 1.0863788
True 1.1055121
True

In [ ]:
# TP	29
# FN	1
# TN	35
# FP	5

# Precision = TP/(TP+FP) = 29/(29+5) = 0.85
# Recall = TP/(TP+FN) = 29/(29+1) = 0.96
# Accuracy = (TP+TN)/(TP+TN+FP+FN) = (29+35)/(29+35+5+1) = 0.92
# F1 = 2*Precision*Recall/(Precision+Recall) = 2*0.85*0.96/(0.85+0.96) = 0.90

